# Applied Data Science Capstone

*determine neighborhoods on the other side of the city that are exactly the same as your current neighborhood*

In [20]:
import pandas as pd
import numpy as np

## Segmenting and Clustering Neighborhoods in Toronto

Download html Wikipedia page

In [10]:
from urllib.request import Request, urlopen
from urllib.error import URLError

data_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

req = Request(data_url)

try:
    with urlopen(req) as response:
        page_bytes = response.read()
        page = page_bytes.decode("utf8")
except URLError as e:
    if hasattr(e, 'reason'):
        print('We failed to reach a server.')
        print('Reason: ', e.reason)
    elif hasattr(e, 'code'):
        print('The server couldn\'t fulfill the request.')
        print('Error code: ', e.code)    
        

Declaring custom html parser. 

In [16]:
from html.parser import HTMLParser

class TableParser(HTMLParser):
    
    def __init__(self):
        
        HTMLParser.__init__(self)
        self._header = False
        self._cell = False
        self.tables = []
        self._current_cell = []
        self._current_row = []
        self._data_separator=' '
        self._current_table = []
        
    def handle_starttag(self, tag, attrs):
        if(tag == 'td'):
            self._cell = True
        if(tag == 'th'):
            self._header = True
            
    def handle_data(self, data):
        if(self._header or self._cell):
            self._current_cell.append(data.strip())
            
    def handle_charref(self, name):
        
        if self._parse_html_entities:
            self.handle_data(self.unescape('&#{};'.format(name)))
            
    def handle_endtag(self, tag):
        
        if tag == 'td':
            self._in_td = False
        elif tag == 'th':
            self._in_th = False

        if tag in ['td', 'th']:
            final_cell = self._data_separator.join(self._current_cell).strip()
            self._current_row.append(final_cell)
            self._current_cell = []
        elif tag == 'tr':
            self._current_table.append(self._current_row)
            self._current_row = []
        elif tag == 'table':
            self.tables.append(self._current_table)
            self._current_table = []

Parsing raw html text to padas dataframe

In [64]:
parser = TableParser()
parser.feed(page)

if len(parser.tables) > 0:
    table = np.array(parser.tables[0])
    df_raw = pd.DataFrame(data=table[1:,0:], columns=table[0,0:])
else:   
    print('Tables not found')

In [47]:
df_raw.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Cleaning dataset:
   * Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
   * More than one neighborhood can exist in one postal code area.
   * If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [77]:
df_raw = df_raw[df_raw.Borough != 'Not assigned']
df_raw.loc[df_raw.Neighbourhood == 'Not assigned', 'Neighbourhood'] = df_raw.loc[df_raw.Neighbourhood == 'Not assigned', 'Borough']
df_raw = df_raw.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x)).reset_index()

In [78]:
df_raw.shape

(103, 3)

## Longitude coordinates of each neighborhood.

Installing geocoder

In [90]:
import sys
!{sys.executable} -m pip install geocoder

     |████████████████████████████████| 102kB 19.7MB/s ta 0:00:01


In [127]:
import geocoder
import json

def  get_coordinates(postcode):
    g = geocoder.arcgis('{}, Toronto'.format(postcode))
    latitude =  g.json['lat']
    longitude = g.json['lng']
    
    return [latitude, longitude]

In [132]:
df_raw['Latitude'] = 0.0
df_raw['Longitude'] = 0.0

for index, row in df_raw.iterrows():
    coord =  get_coordinates(row['Postcode'])
    df_raw.at[index,'Latitude'] = coord[0]
    df_raw.at[index,'Longitude'] = coord[1]

df_raw.head()        

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785730,-79.158750
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175256
3,M1G,Scarborough,Woburn,43.768359,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440
